In [53]:
import openai

from typing import Optional, Union
import json

# def wrapped_chat_gpt_completion(
#     messages: list,
#     history: Optional[list] = [],
#     max_tokens: Optional[int] = 16,
#     temperature: Optional[float] = 1,
#     top_p: Optional[float] = 1,
#     n: Optional[int] = 1,
#     frequency_penalty: Optional[float] = 0,
#     stop: Optional[Union[str, list]] = None,
# ) -> str:
#     """
#     Generates a completion using the GPT-3.5-turbo model.

#     messages needs to be in the format:
#     [
#         {
#             "role": "user",
#             "content": "Hello, how are you?"
#         },
#         {
#             "role": "assistant",
#             "content": "I am doing well, how about you?"
#         }
#         ...
#     ]
#     """
#     if history:
#         messages = history + messages

#     response = openai.ChatCompletion.create(
#         model='gpt-3.5-turbo',
#         messages=messages,
#         max_tokens=max_tokens,
#         temperature=temperature,
#         top_p=top_p,
#         n=n,
#         frequency_penalty=frequency_penalty,
#         stop=stop,
#     )
#     if response is None or response["choices"] is None or len(response["choices"]) == 0:
#         return ""

#     choices = response["choices"]
#     top_choice = choices[0]
#     preview = top_choice["message"]["content"].strip()

#     messages = messages + [{"role": "assistant", "content": preview}]
#     return messages, preview


In [54]:

import re
from pathlib import Path
import time
import os
import sys
import random
from dotenv import load_dotenv
env_path = Path('../..') / '.env'
load_dotenv(dotenv_path=env_path)

print(os.getcwd())
sys.path.append('/Users/hunter/Desktop/SellScale/sellscale-api/notebooks/gpt_api')

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
def attempt_chat_completion(
    messages: list,
    max_tokens: Optional[int] = 16,
    temperature: Optional[float] = 1,
    top_p: Optional[float] = 1,
    n: Optional[int] = 1,
    frequency_penalty: Optional[float] = 0,
    stop: Optional[Union[str, list]] = None,
    model: str = 'gpt-3.5-turbo',
    max_attempts: int = 3,
):
    attempts = 0
    exception = None
    while attempts < max_attempts:
        try:
            response = openai.ChatCompletion.create(
                model='gpt-4',
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
                top_p=top_p,
                n=n,
                frequency_penalty=frequency_penalty,
                stop=stop,
            )
            return response
        except Exception as e:
            attempts += 1
            # Add a random delay before the next attempt
            exception = e
            time.sleep(random.uniform(0.5, 1.5))
    raise Exception(exception)


def wrapped_chat_gpt_completion(
    messages: list,
    max_tokens: Optional[int] = 16,
    temperature: Optional[float] = 1,
    top_p: Optional[float] = 1,
    n: Optional[int] = 1,
    frequency_penalty: Optional[float] = 0,
    stop: Optional[Union[str, list]] = None,
    model: str = 'gpt-4',
    max_attempts: int = 3,
    tools: Optional[list] = None,
) -> str:
    """
    Generates a completion using a GPT model.

    messages needs to be in the format:
    [
        {
            "role": "user",
            "content": "Hello, how are you?"
        },
        {
            "role": "bot",
            "content": "I am doing well, how about you?"
        }
        ...
    ]
    """

    # OpenAI
    response = attempt_chat_completion(
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        n=n,
        frequency_penalty=frequency_penalty,
        stop=stop,
        model=model,
        max_attempts=max_attempts,
    )
    if response is None or response["choices"] is None or len(response["choices"]) == 0:
        return ""

    choices = response["choices"]
    top_choice = choices[0]
    preview = top_choice["message"]["content"].strip()
    return preview



/Users/hunter/Desktop/sellscale/sellscale-api/notebooks/gpt_api


In [179]:
def chat_ai_verify_scheduling_convo(messages: list[str], seller: str, current_status: str, use_previous = False) -> bool:
    """Verifies if the conversation is about scheduling a meeting.

    Args:
        messages (list[str]): The conversation history.
        seller (str): The name of the seller.

    Returns:
        bool: Whether the conversation is about scheduling a meeting.
    """
    # Construct the transcript
    transcript = ""
    for message in messages:
        transcript += message + "\n\n"

    previous_prompt = """The following transcript was determined to feature a seller and a potential customer discussing a time to meet. The following transcript was thus classified as "SCHEDULING".

Can you confirm, by replying either 0 (for False) or 1 (for True) that this conversation meets the following criteria for "SCHEDULING."

Criteria (one or the other):
1. Both parties are actively engaged in finding a time to meet.
2. The customer is showing a willingness to schedule a call.

Seller: {seller_name}

--- Start Transcript ---
{transcript}
--- End Transcript ---
""".format(
        seller_name=seller, transcript=transcript
    )

    prompt = """The following transcript was determined to feature a seller and a potential customer discussing a time to meet. The following transcript was thus classified as "SCHEDULING".

Can you confirm, by replying either 0 (for False) or 1 (for True) that this conversation meets the following criteria for "SCHEDULING."

Criteria:
The two parties are showing a willingness to schedule a meeting.
Note: The scheduling must be unresolved to be true-- be careful. people might be suggesting times and places that work without having confirmed.
Make sure that both parties have not settled on when and where they will be seeing each other, and that there are hanging questions.

If the CURRENT_STATUS is DEMO_SET, please automatically return 0.

Seller: {seller_name}
CURRENT_STATUS: {current_status}

--- Start Transcript ---
{transcript}
--- End Transcript ---

""".format(
        seller_name=seller, transcript=transcript, current_status=current_status
    )

    if (use_previous):
        prompt = previous_prompt

    response = get_text_generation(
        [{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=10,
        model='gpt-4',
        type="MISC_CLASSIFY",
    )

    match = re.search(r"\d+", response)
    if match:
        number = int(match.group(0))
    else:
        return False

    return number == 1
def get_text_generation(
    messages: list,
    type: str,
    model: str,
    max_tokens: int,
    prospect_id: Optional[int] = None,
    client_sdr_id: Optional[int] = None,
    temperature: Optional[float] = 1,
    use_cache: bool = False,
    tools: Optional[list] = None,
    stream_event: Optional[str] = None,
    stream_room_id: Optional[str] = None,
) -> Optional[str]:
    # type = "LI_MSG_INIT" | "LI_MSG_OTHER" | "RESEARCH" | "EMAIL" | "VOICE_MSG" | "ICP_CLASSIFY"
    # | "TEXT_EDITOR" | "MISC_CLASSIFY" | "MISC_SUMMARIZE" | "LI_CTA" | "CLIENT_ASSETS" | "SEQUENCE_GEN_<ID>"

    def normalize_string(string: str) -> str:
        string = re.sub(r"\\n", " ", string)
        string = string.strip()  # Remove leading/trailing whitespace
        string = " ".join(string.split())
        return string

    try:
        json_msgs = normalize_string(json.dumps(messages))
    except Exception as e:
        json_msgs = None

    text_gen = None
    # Handle streaming

    # Handle normal completion
    if json_msgs and text_gen:
        return text_gen.completion
    else:
        response = wrapped_chat_gpt_completion(
            messages,
            max_tokens=max_tokens,
            temperature=temperature,
        )
        if True:
            return response


        else:
            return ""



In [180]:
# chat_ai_verify_scheduling_convo(transcripts[0], 'Hunter Gordon', 'DEMO_SET')

In [181]:
transcripts = [
    [
        "Customer (10/02/2023, 14:32:10): Hi Hunter, I'm interested in the product we discussed earlier. When would be a good time to meet and discuss further?",
        "Hunter Gordon (10/02/2023, 14:35:22): Hi! Great to hear you're interested. Would you be available tomorrow at 3 PM?",
        "Customer (10/02/2023, 14:36:08): Tomorrow at 3 PM works fine for me. Where should we meet?",
        "Hunter Gordon (10/02/2023, 14:37:45): Let's meet at the downtown coffee shop. Does that location work for you?",
        "Customer (10/02/2023, 14:39:10): Yes, that works. See you then!",
        "Hunter Gordon (10/02/2023, 14:40:22): Perfect, see you tomorrow at 3 PM at the downtown coffee shop.",
        "Customer (10/02/2023, 14:41:15): Thanks, looking forward to it!"
    ],
    [
        "Customer (10/03/2023, 09:12:34): Morning Hunter, I'm available to meet on Wednesday. Does 11 AM work for you?",
        "Hunter Gordon (10/03/2023, 09:15:56): Good morning! 11 AM on Wednesday is perfect. Shall we meet at your office?",
        "Customer (10/03/2023, 09:17:23): Yes, my office works. See you then.",
        "Hunter Gordon (10/03/2023, 09:18:09): Great, see you on Wednesday at 11 AM."
    ],
    [
        "Customer (10/02/2023, 15:22:10): Hi Hunter, can we meet this week to discuss the project details?",
        "Hunter Gordon (10/02/2023, 15:25:40): Sure, how about Friday afternoon?",
        "Customer (10/02/2023, 15:26:45): Friday works for me. Can we say 2 PM?",
        "Hunter Gordon (10/02/2023, 15:27:30): 2 PM on Friday it is. Where would you like to meet?",
        "Customer (10/02/2023, 15:28:15): How about the central park cafe?",
        "Hunter Gordon (10/02/2023, 15:29:00): Sounds good! See you at the central park cafe at 2 PM on Friday."
    ],
    [
        "Customer (10/01/2023, 11:02:00): Hello Hunter, when would be a good time to meet and discuss the proposal?",
        "Hunter Gordon (10/01/2023, 11:05:10): Hello! I'm free this Thursday. Does 4 PM suit you?",
        "Customer (10/01/2023, 11:06:22): Yes, 4 PM on Thursday is fine. Where should we meet?",
        "Hunter Gordon (10/01/2023, 11:07:45): Let's go with the library conference room. Is that okay?",
        "Customer (10/01/2023, 11:08:30): Yes, perfect. See you on Thursday at 4 PM.",
        "Hunter Gordon (10/01/2023, 11:09:10): Looking forward to it!"
    ],
    [
        "Customer (10/04/2023, 14:44:12): Hi Hunter, are you available to meet next Tuesday afternoon?",
        "Hunter Gordon (10/04/2023, 14:47:18): Hi! Next Tuesday works. How about 1 PM?",
        "Customer (10/04/2023, 14:48:20): 1 PM is good. Shall we meet at the co-working space?",
        "Hunter Gordon (10/04/2023, 14:49:30): Yes, the co-working space sounds perfect. See you then.",
        "Customer (10/04/2023, 14:50:22): Great, see you on Tuesday at 1 PM."
    ],
    [
        "Customer (10/05/2023, 10:15:40): Hello Hunter, can we schedule a meeting for this Friday?",
        "Hunter Gordon (10/05/2023, 10:17:55): Hi! Yes, we can. Does 10 AM work for you?",
        "Customer (10/05/2023, 10:19:12): Yes, 10 AM works. Where shall we meet?",
        "Hunter Gordon (10/05/2023, 10:20:30): How about the new coffee shop on Elm Street?",
        "Customer (10/05/2023, 10:21:45): Perfect! See you there at 10 AM on Friday.",
        "Hunter Gordon (10/05/2023, 10:22:33): See you then!"
    ],
    [
        "Customer (10/06/2023, 16:03:10): Hi Hunter, when can we meet to discuss the contract?",
        "Hunter Gordon (10/06/2023, 16:05:20): Hi! I have some free time on Monday. Does 9 AM work?",
        "Customer (10/06/2023, 16:06:15): Yes, 9 AM on Monday works for me. Where should we meet?",
        "Hunter Gordon (10/06/2023, 16:07:45): Let's meet at the office cafeteria.",
        "Customer (10/06/2023, 16:08:30): Sounds good. See you then!",
        "Hunter Gordon (10/06/2023, 16:09:22): Looking forward to it!"
    ],
    [
        "Customer (10/07/2023, 13:32:10): Hi Hunter, can we meet this weekend to finalize the plans?",
        "Hunter Gordon (10/07/2023, 13:35:45): Hi! How about Saturday at 11 AM?",
        "Customer (10/07/2023, 13:37:05): That works. Where should we meet?",
        "Hunter Gordon (10/07/2023, 13:38:22): Let's meet at the seaside cafe.",
        "Customer (10/07/2023, 13:39:10): Sure, see you at 11 AM on Saturday.",
        "Hunter Gordon (10/07/2023, 13:40:15): See you then!"
    ],
    [
        "Customer (10/08/2023, 17:23:16): Hi Hunter, are you available to meet tomorrow?",
        "Hunter Gordon (10/08/2023, 17:26:30): Hi! Yes, I'm free. What time works for you?",
        "Customer (10/08/2023, 17:27:45): How about 10 AM?",
        "Hunter Gordon (10/08/2023, 17:28:50): 10 AM is good. Let's meet at the office lobby.",
        "Customer (10/08/2023, 17:29:30): See you there!",
        "Hunter Gordon (10/08/2023, 17:30:40): Looking forward to it!"
    ],
    [
        "Customer (10/09/2023, 12:15:30): Hi Hunter, can we discuss the project over a call?",
        "Hunter Gordon (10/09/2023, 12:18:45): Hi! Sure, when would you like to have the call?",
        "Customer (10/09/2023, 12:20:00): Can we do it tomorrow around 3 PM?",
        "Hunter Gordon (10/09/2023, 12:21:15): Tomorrow at 3 PM works for me. Let's do it.",
        "Customer (10/09/2023, 12:22:30): Great, talk to you then!",
        "Hunter Gordon (10/09/2023, 12:23:40): Talk to you then!",
        "Customer (10/09/2023, 12:24:50): By the way, do you have the latest project report?",
        "Hunter Gordon (10/09/2023, 12:26:00): Yes, I will send it to you before our call."
    ],
    [
        "Customer (10/10/2023, 08:30:00): Hi Hunter, can we have a video conference this week?",
        "Hunter Gordon (10/10/2023, 08:33:15): Hi! Sure, when would you prefer?",
        "Customer (10/10/2023, 08:34:30): How about Thursday at 1 PM?",
        "Hunter Gordon (10/10/2023, 08:35:45): Thursday at 1 PM is fine. Let's have the video conference then.",
        "Customer (10/10/2023, 08:36:50): Perfect, see you on Thursday!",
        "Hunter Gordon (10/10/2023, 08:38:00): See you on Thursday!"
    ],
]

In [182]:
active_scheduling_transcripts = [
    [
        "Customer (10/02/2023, 14:32:10): Hi Hunter, I'm interested in the product we discussed earlier. When would be a good time to meet and discuss further?",
        "Hunter Gordon (10/02/2023, 14:35:22): Hi! Great to hear you're interested. Would you be available tomorrow at 3 PM?",
        "Customer (10/02/2023, 14:36:08): Tomorrow at 3 PM works fine for me. Where should we meet?",
        "Hunter Gordon (10/02/2023, 14:37:45): Let's meet at the downtown coffee shop. Does that location work for you?"
    ],
    [
        "Customer (10/03/2023, 09:12:34): Morning Hunter, I'm available to meet on Wednesday. Does 11 AM work for you?",
        "Hunter Gordon (10/03/2023, 09:15:56): Good morning! 11 AM on Wednesday is perfect. Shall we meet at your office?"
    ],
    [
        "Customer (10/02/2023, 15:22:10): Hi Hunter, can we meet this week to discuss the project details?",
        "Hunter Gordon (10/02/2023, 15:25:40): Sure, how about Friday afternoon?",
        "Customer (10/02/2023, 15:26:45): Friday works for me. Can we say 2 PM?",
        "Hunter Gordon (10/02/2023, 15:27:30): 2 PM on Friday it is. Where would you like to meet?"
    ],
    [
        "Customer (10/01/2023, 11:02:00): Hello Hunter, when would be a good time to meet and discuss the proposal?",
        "Hunter Gordon (10/01/2023, 11:05:10): Hello! I'm free this Thursday. Does 4 PM suit you?",
        "Customer (10/01/2023, 11:06:22): Yes, 4 PM on Thursday is fine. Where should we meet?"
    ],
    [
        "Customer (10/04/2023, 14:44:12): Hi Hunter, are you available to meet next Tuesday afternoon?",
        "Hunter Gordon (10/04/2023, 14:47:18): Hi! Next Tuesday works. How about 1 PM?",
        "Customer (10/04/2023, 14:48:20): 1 PM is good. Shall we meet at the co-working space?"
    ],
    [
        "Customer (10/05/2023, 10:15:40): Hello Hunter, can we schedule a meeting for this Friday?",
        "Hunter Gordon (10/05/2023, 10:17:55): Hi! Yes, we can. Does 10 AM work for you?",
        "Customer (10/05/2023, 10:19:12): Yes, 10 AM works. Where shall we meet?"
    ],
    [
        "Customer (10/06/2023, 16:03:10): Hi Hunter, when can we meet to discuss the contract?",
        "Hunter Gordon (10/06/2023, 16:05:20): Hi! I have some free time on Monday. Does 9 AM work?",
        "Customer (10/06/2023, 16:06:15): Yes, 9 AM on Monday works for me. Where should we meet?"
    ],
    [
        "Customer (10/07/2023, 13:32:10): Hi Hunter, can we meet this weekend to finalize the plans?",
        "Hunter Gordon (10/07/2023, 13:35:45): Hi! How about Saturday at 11 AM?",
        "Customer (10/07/2023, 13:37:05): That works. Where should we meet?"
    ],
    [
        "Customer (10/08/2023, 17:23:16): Hi Hunter, are you available to meet tomorrow?",
        "Hunter Gordon (10/08/2023, 17:26:30): Hi! Yes, I'm free. What time works for you?",
        "Customer (10/08/2023, 17:27:45): How about 10 AM? Where should we meet?"
    ],
    [
        "Customer (10/09/2023, 12:15:30): Hi Hunter, can we discuss the project over a call?",
        "Hunter Gordon (10/09/2023, 12:18:45): Hi! Sure, when would you like to have the call?",
        "Customer (10/09/2023, 12:20:00): Can we do it tomorrow around 3 PM?",
    ],
    [
        "Customer (10/10/2023, 08:30:00): Hi Hunter, can we have a video conference this week?",
        "Hunter Gordon (10/10/2023, 08:33:15): Hi! Sure, when would you prefer?",
        "Customer (10/10/2023, 08:34:30): How about Thursday at 1 PM?"
    ],
]

In [183]:
import pandas as pd
from IPython.display import display, HTML

name = 'Hunter Gordon'
demo_set = 'DEMO_SET'

def generate_truth_table(transcripts, name, demo_set, use_previous=False):
    results = []
    for transcript in transcripts:
        result = chat_ai_verify_scheduling_convo(transcript, name, demo_set, use_previous)
        results.append(result)
    
    truth_table = pd.DataFrame({
        'Transcript': [f'Transcript {i+1}' for i in range(len(transcripts))],
        'Are they scheduling?': results,
        'Current Status?': demo_set,
        'Previous Prompt': [use_previous] * len(transcripts)
    })
    
    return truth_table

def display_pretty_table(truth_table):
    display(HTML(truth_table.to_html(classes='table table-striped table-bordered table-hover')))

# Example usage with DEMO_SET
truth_table_demo_set = generate_truth_table(transcripts, name, demo_set)
truth_table_demo_set_previous = generate_truth_table(transcripts, name, demo_set, True)
combined_truth_table_demo_set = pd.concat([truth_table_demo_set, truth_table_demo_set_previous], axis=1)
display_pretty_table(combined_truth_table_demo_set)

# Example usage with ACTIVE_CONVO_SCHEDULING
demo_set_active_convo = 'ACTIVE_CONVO_SCHEDULING'
truth_table_active_convo = generate_truth_table(transcripts, name, demo_set_active_convo)
truth_table_active_convo_previous = generate_truth_table(transcripts, name, demo_set_active_convo, True)
combined_truth_table_active_convo = pd.concat([truth_table_active_convo, truth_table_active_convo_previous], axis=1)
display_pretty_table(combined_truth_table_active_convo)

print('Transcripts where they are trying to schedule a meeting')

# Example usage with DEMO_SET
truth_table_demo_set = generate_truth_table(active_scheduling_transcripts, name, demo_set)
truth_table_demo_set_previous = generate_truth_table(active_scheduling_transcripts, name, demo_set, True)
combined_truth_table_demo_set = pd.concat([truth_table_demo_set, truth_table_demo_set_previous], axis=1)
display_pretty_table(combined_truth_table_demo_set)

# Example usage with ACTIVE_CONVO_SCHEDULING
truth_table_active_convo = generate_truth_table(active_scheduling_transcripts, name, demo_set_active_convo)
truth_table_active_convo_previous = generate_truth_table(active_scheduling_transcripts, name, demo_set_active_convo, True)
combined_truth_table_active_convo = pd.concat([truth_table_active_convo, truth_table_active_convo_previous], axis=1)
display_pretty_table(combined_truth_table_active_convo)


,Transcript,Are they scheduling?,Current Status?,Previous Prompt,Transcript,Are they scheduling?,Current Status?,Previous Prompt
0,Transcript 1,False,DEMO_SET,False,Transcript 1,True,DEMO_SET,True
1,Transcript 2,False,DEMO_SET,False,Transcript 2,True,DEMO_SET,True
2,Transcript 3,False,DEMO_SET,False,Transcript 3,True,DEMO_SET,True
3,Transcript 4,False,DEMO_SET,False,Transcript 4,True,DEMO_SET,True
4,Transcript 5,False,DEMO_SET,False,Transcript 5,True,DEMO_SET,True
5,Transcript 6,False,DEMO_SET,False,Transcript 6,True,DEMO_SET,True
6,Transcript 7,False,DEMO_SET,False,Transcript 7,True,DEMO_SET,True
7,Transcript 8,False,DEMO_SET,False,Transcript 8,True,DEMO_SET,True
8,Transcript 9,False,DEMO_SET,False,Transcript 9,True,DEMO_SET,True
9,Transcript 10,False,DEMO_SET,False,Transcript 10,True,DEMO_SET,True


,Transcript,Are they scheduling?,Current Status?,Previous Prompt,Transcript,Are they scheduling?,Current Status?,Previous Prompt
0,Transcript 1,False,ACTIVE_CONVO_SCHEDULING,False,Transcript 1,True,ACTIVE_CONVO_SCHEDULING,True
1,Transcript 2,False,ACTIVE_CONVO_SCHEDULING,False,Transcript 2,True,ACTIVE_CONVO_SCHEDULING,True
2,Transcript 3,False,ACTIVE_CONVO_SCHEDULING,False,Transcript 3,True,ACTIVE_CONVO_SCHEDULING,True
3,Transcript 4,False,ACTIVE_CONVO_SCHEDULING,False,Transcript 4,True,ACTIVE_CONVO_SCHEDULING,True
4,Transcript 5,False,ACTIVE_CONVO_SCHEDULING,False,Transcript 5,True,ACTIVE_CONVO_SCHEDULING,True
5,Transcript 6,False,ACTIVE_CONVO_SCHEDULING,False,Transcript 6,True,ACTIVE_CONVO_SCHEDULING,True
6,Transcript 7,False,ACTIVE_CONVO_SCHEDULING,False,Transcript 7,True,ACTIVE_CONVO_SCHEDULING,True
7,Transcript 8,False,ACTIVE_CONVO_SCHEDULING,False,Transcript 8,True,ACTIVE_CONVO_SCHEDULING,True
8,Transcript 9,False,ACTIVE_CONVO_SCHEDULING,False,Transcript 9,True,ACTIVE_CONVO_SCHEDULING,True
9,Transcript 10,False,ACTIVE_CONVO_SCHEDULING,False,Transcript 10,True,ACTIVE_CONVO_SCHEDULING,True


Transcripts where they are trying to schedule a meeting


,Transcript,Are they scheduling?,Current Status?,Previous Prompt,Transcript,Are they scheduling?,Current Status?,Previous Prompt
0,Transcript 1,False,DEMO_SET,False,Transcript 1,True,DEMO_SET,True
1,Transcript 2,False,DEMO_SET,False,Transcript 2,True,DEMO_SET,True
2,Transcript 3,False,DEMO_SET,False,Transcript 3,True,DEMO_SET,True
3,Transcript 4,False,DEMO_SET,False,Transcript 4,True,DEMO_SET,True
4,Transcript 5,False,DEMO_SET,False,Transcript 5,True,DEMO_SET,True
5,Transcript 6,False,DEMO_SET,False,Transcript 6,True,DEMO_SET,True
6,Transcript 7,False,DEMO_SET,False,Transcript 7,True,DEMO_SET,True
7,Transcript 8,False,DEMO_SET,False,Transcript 8,True,DEMO_SET,True
8,Transcript 9,False,DEMO_SET,False,Transcript 9,True,DEMO_SET,True
9,Transcript 10,False,DEMO_SET,False,Transcript 10,True,DEMO_SET,True


,Transcript,Are they scheduling?,Current Status?,Previous Prompt,Transcript,Are they scheduling?,Current Status?,Previous Prompt
0,Transcript 1,True,ACTIVE_CONVO_SCHEDULING,False,Transcript 1,True,ACTIVE_CONVO_SCHEDULING,True
1,Transcript 2,True,ACTIVE_CONVO_SCHEDULING,False,Transcript 2,True,ACTIVE_CONVO_SCHEDULING,True
2,Transcript 3,True,ACTIVE_CONVO_SCHEDULING,False,Transcript 3,True,ACTIVE_CONVO_SCHEDULING,True
3,Transcript 4,True,ACTIVE_CONVO_SCHEDULING,False,Transcript 4,True,ACTIVE_CONVO_SCHEDULING,True
4,Transcript 5,True,ACTIVE_CONVO_SCHEDULING,False,Transcript 5,True,ACTIVE_CONVO_SCHEDULING,True
5,Transcript 6,True,ACTIVE_CONVO_SCHEDULING,False,Transcript 6,True,ACTIVE_CONVO_SCHEDULING,True
6,Transcript 7,True,ACTIVE_CONVO_SCHEDULING,False,Transcript 7,True,ACTIVE_CONVO_SCHEDULING,True
7,Transcript 8,True,ACTIVE_CONVO_SCHEDULING,False,Transcript 8,True,ACTIVE_CONVO_SCHEDULING,True
8,Transcript 9,True,ACTIVE_CONVO_SCHEDULING,False,Transcript 9,True,ACTIVE_CONVO_SCHEDULING,True
9,Transcript 10,True,ACTIVE_CONVO_SCHEDULING,False,Transcript 10,True,ACTIVE_CONVO_SCHEDULING,True


In [126]:
transcripts

[["Customer (10/02/2023, 14:32:10): Hi Hunter, I'm interested in the product we discussed earlier. When would be a good time to meet and discuss further?",
  "Hunter Gordon (10/02/2023, 14:35:22): Hi! Great to hear you're interested. Would you be available tomorrow at 3 PM?",
  'Customer (10/02/2023, 14:36:08): Tomorrow at 3 PM works fine for me. Where should we meet?',
  "Hunter Gordon (10/02/2023, 14:37:45): Let's meet at the downtown coffee shop. Does that location work for you?",
  'Customer (10/02/2023, 14:39:10): Yes, that works. See you then!',
  'Hunter Gordon (10/02/2023, 14:40:22): Perfect, see you tomorrow at 3 PM at the downtown coffee shop.',
  'Customer (10/02/2023, 14:41:15): Thanks, looking forward to it!'],
 ["Customer (10/03/2023, 09:12:34): Morning Hunter, I'm available to meet on Wednesday. Does 11 AM work for you?",
  'Hunter Gordon (10/03/2023, 09:15:56): Good morning! 11 AM on Wednesday is perfect. Shall we meet at your office?',
  'Customer (10/03/2023, 09:17:23